In [1]:
import json
import requests
import popelines
import os
from datetime import datetime, timedelta
from main import fix_values, populating_vals, flatten, flatten_dupe_vals

In [2]:
from generate_original import read_endpoint, generate_projects


In [3]:
# from batch_update_projects import check_bq_ts, check_api_ts, generate_new_entity

ModuleNotFoundError: No module named 'batch_update_projects'

In [4]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}


# Fuctions

In [ ]:
def check_bq_ts(table_name, ts_col_name):
    last_upload_ts = pope.find_last_entry(table_name, ts_col_name) + timedelta(hours=-8)
    return last_upload_ts

In [ ]:
# this function should get timestamp for each entity (project, experiment, or result), from API,
# returns only those with a timestamp that is later than the benchmark, benchmark_ts
def check_api_ts(api_path, table, ts_col, benchmark_ts, existing_id):
    
    # compare each of the project's last_modified timestamp to the above last upload_ts
    # select only if last_modified > upload_ts
    updating_entity_id = []
    if table == 'project':
        all_existing = generate_projects(api_path, headers)
    # elif table == 'experiment':
    #     all_existing = generate_experiment(api_path, headers)
    else:
        pass

    for entity in all_existing:
        entity_ts = datetime.strptime(entity[ts_col], '%Y-%m-%dT%H:%M:%S.%fz') + timedelta(hours=-8)
        if  entity_ts > benchmark_ts:
            updating_entity_id.append(entity['id'])
        elif entity['id'] not in existing_id:
            updating_entity_id.append(entity['id'])

    return updating_entity_id

In [ ]:
def generate_new_entity(id_list, api_path, table):
    new_json = []
    for entity_id in id_list:
        if table == 'project':
            entity_info = generate_projects(f'{api_path}/{entity_id}', headers)
            entity_info['upload_ts'] = str(datetime.utcnow())

        # need to add experiment table
        else:
            pass

        new_json.append(entity_info)

    return new_json

# Existing experiments

In [16]:
experiment_query = open('existing_experiments.sql').read()

In [17]:
pope.bq_query(experiment_query)

[Row((10800885728, datetime.datetime(2019, 2, 21, 12, 29, 6, 527360, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((10622090898, datetime.datetime(2019, 2, 21, 12, 29, 11, 171080, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((10898400714, datetime.datetime(2019, 2, 21, 12, 29, 16, 241720, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((12169545706, datetime.datetime(2019, 2, 21, 12, 29, 11, 191284, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((12096030597, datetime.datetime(2019, 2, 21, 12, 29, 11, 190382, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((10756790433, datetime.datetime(2019, 2, 21, 12, 29, 6, 524103, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((12239002639, datetime.datetime(2019, 2, 21, 12, 29, 11, 193198, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((11378763890, datetime.datetime(2019, 2, 21, 12, 29, 11, 185497, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((10702922345, datetime.datetime(2019, 2, 21, 12, 29, 16, 232393, tzinfo=<UTC>)), {'id': 0, 'f0_': 1}),
 Row((11191905720, datetime.da

In [25]:
existing_experiments = []
for result in pope.bq_query(experiment_query):
    existing_experiments.append((result[0], datetime.strftime(result[1], '%Y-%m-%dT%H:%M:%S.%fz')))
    

In [53]:
existing_experiments

[(10800885728, '2019-02-21T12:29:06.527360z'),
 (10622090898, '2019-02-21T12:29:11.171080z'),
 (10898400714, '2019-02-21T12:29:16.241720z'),
 (12169545706, '2019-02-21T12:29:11.191284z'),
 (12096030597, '2019-02-21T12:29:11.190382z'),
 (10756790433, '2019-02-21T12:29:06.524103z'),
 (12239002639, '2019-02-21T12:29:11.193198z'),
 (11378763890, '2019-02-21T12:29:11.185497z'),
 (10702922345, '2019-02-21T12:29:16.232393z'),
 (11191905720, '2019-02-21T12:29:11.182279z'),
 (10488931082, '2019-02-21T12:29:11.169689z'),
 (10564323775, '2019-02-21T12:29:11.170463z'),
 (10731441129, '2019-02-21T12:29:16.235035z'),
 (12632940119, '2019-02-21T12:29:12.874823z'),
 (11367191147, '2019-02-21T12:29:11.185347z'),
 (11556750451, '2019-02-21T12:29:11.188422z'),
 (10683631399, '2019-02-21T12:29:11.173833z'),
 (12619380698, '2019-02-21T12:29:16.249113z'),
 (12903040293, '2019-02-21T12:29:11.194462z'),
 (10945903729, '2019-02-21T12:29:06.528197z'),
 (11087580674, '2019-02-21T12:29:11.180814z'),
 (11095671052

In [54]:
10945903729 in [x[0] for x in existing_experiments]

True

In [63]:
[x[1] for x in existing_experiments if x[0] == 12903040293]

['2019-02-21T12:29:11.194462z']

# Get all experiments

In [30]:
project_query = open('existing_projects.sql').read()
existing_projects = []
for result in pope.bq_query(project_query):
    existing_projects.append(result[0])

In [78]:
existing_projects

[10429921250,
 10849384554,
 10347872424,
 9965963792,
 10427612860,
 13655420480,
 10057253317]

In [33]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'

In [83]:
updated_experiments = []
for project_id in existing_projects:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)

    for exp in exp_list:
        if exp['id'] not in [x[0] for x in existing_experiments]:
            updated_experiments.append(exp)
            
        else:
            if exp['last_modified'] > [x[1] for x in existing_experiments if x[0] == exp['id']][0]:
                updated_experiments.append(exp)
                
            else:
                None
        
        

In [86]:
updated_experiments

[{'allocation_policy': 'manual',
  'audience_conditions': '["and", {"audience_id": 13703930196}]',
  'campaign_id': 13726600170,
  'changes': [],
  'created': '2019-02-25T16:57:59.156407Z',
  'holdback': 0,
  'id': 13727000072,
  'is_classic': False,
  'last_modified': '2019-02-25T17:06:46.997755Z',
  'metrics': [],
  'name': "Lead Form Placement on Mobile - Feb '19",
  'project_id': 10427612860,
  'status': 'not_started',
  'type': 'a/b',
  'url_targeting': {'activation_type': 'immediate',
   'conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "https://pages.infusionsoft.com/ibk-2019_strategic_planning_kit_gate.html"}]]',
   'edit_url': 'https://pages.infusionsoft.com/ibk-2019_strategic_planning_kit_gate.html',
   'key': '10427612860_url_targeting_for_lead_form_placement_on_mobile__feb',
   'page_id': 13734660063},
  'variations': [{'actions': [],
    'archived': False,
    'name': 'Form below description (Control)',
    'status': 'active',
    'variation_id

In [89]:
from generate_original import generate_experiments

In [101]:
def generate_experiments(exp_list):
    ###### Game plan is to separate nested fields from single layer fields, upload them to separate table, then do joins on BQ level
    all_singles = []
    metrics_table = []
    variations_table = []

    for exp in exp_list:
        print(f"Processing experiment {exp['id']}")

        # single layer fields:
        nested_key_list = []
        for k,v in exp.items():
            if isinstance(v, list):
                nested_key_list.append(k)
        
        single_layer_experiment = {}    
        for k,v in exp.items():
            if k not in nested_key_list:
                k = k.replace('-', '_')
                single_layer_experiment[k] = exp[k]
        single_layer_experiment['upload_ts'] = str(datetime.utcnow())

        all_singles.append(flatten(single_layer_experiment, {}, ''))

        # nested part into separate tables:
        # metrics table:

        flattened_metric = []
        for element in exp['metrics']:
            flattened_metric.append(element)

        updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
        new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

        metric_list = []
        for metric in new_flattened_metric:
            metric_dict = {}
            metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
            if 'metrics_event_id' in metric.keys():
                metric_dict['metrics_event_id'] = metric['metrics_event_id']
            metric_dict['metrics_scope'] =  metric['metrics_scope']
            metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
            metric_dict['experiment_id'] = exp['id']
            metric_dict['upload_ts'] = str(datetime.utcnow())
            metric_list.append(metric_dict)

        metrics_table.extend(metric_list)
    
        # variations table:
        variations = {}
        variations['experiment_id'] = exp['id']
        variations['variations'] = exp['variations']
        variations['upload_ts'] = str(datetime.utcnow())

        flattened_variations = []

        for var in exp['variations']:
            flattened_actions = []  
            
            if len(var['actions']) > 0:
                for action in var['actions']:
                    flattened_changes = []

                    if action['changes'] != []:
                        for element in action['changes']:
                            flattened_changes.append(element)
                        # Replace old 'changes' with new 'flattened_changes'
                        updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                        new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                        update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                        flat = flatten_dupe_vals(vals=update_actions, key='actions')
                        flattened_actions.extend(flat)

                    else:
                        new_actions = flatten_dupe_vals(vals=var, key='actions')
                        flattened_actions.extend(new_actions)
                    
            else:
                other_flat = {}
                for k,v in var.items():
                    if k != 'actions':
                        other_flat['actions'] = []
                        other_flat[k] = v
                flat = [other_flat]
                flattened_actions.extend(flat)

            update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
            flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

        variations_table.extend(flattened_variations)

    return all_singles, metrics_table, variations_table

In [102]:
all_singles, metrics_table, variations_table = generate_experiments(updated_experiments)

Processing experiment 13727000072
Processing experiment 13644120221
Processing experiment 10195324920
Processing experiment 10890302148


In [104]:
pope.write_to_json(file_name='../uploads/update_experiments_single_fields.json', jayson=all_singles, mode='w')
pope.write_to_bq(table_name='update_experiments_single_fields', file_name='../uploads/update_experiments_single_fields.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)

pope.write_to_json(file_name='../uploads/update_experiments_metrics_table.json', jayson=metrics_table, mode='w')
pope.write_to_bq(table_name='update_experiments_metrics_table', file_name='../uploads/update_experiments_metrics_table.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
 
pope.write_to_json(file_name='../uploads/update_experiments_variations_table.json', jayson=variations_table, mode='w')
pope.write_to_bq(table_name='update_experiments_variations_table', file_name='../uploads/update_experiments_variations_table.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)




INFO:root:Writing update_experiments_single_fields to BQ from file ../uploads/update_experiments_single_fields.json


2019-02-25 16:43:02,002 - root - Writing update_experiments_single_fields to BQ from file ../uploads/update_experiments_single_fields.json


INFO:root:Writing update_experiments_metrics_table to BQ from file ../uploads/update_experiments_metrics_table.json


2019-02-25 16:43:05,026 - root - Writing update_experiments_metrics_table to BQ from file ../uploads/update_experiments_metrics_table.json


INFO:root:Writing update_experiments_variations_table to BQ from file ../uploads/update_experiments_variations_table.json


2019-02-25 16:43:08,372 - root - Writing update_experiments_variations_table to BQ from file ../uploads/update_experiments_variations_table.json


# Updating new/modified experiments

In [ ]:
# get the timestamp from bq for experiment table, at which the job was most recently run
last_upload_ts = check_bq_ts('origin_experiments_single_fields', 'upload_ts')



In [ ]:
# get experiments with last_modified timestamps that are later than the previous ts
# getting ready to upload
updating_experiments = check_api_ts(api_path=experiment_endpoint, table='experiment', ts_col='last_modified', benchmark_ts=last_upload_ts.replace(tzinfo=None))
updating_experiments_json = generate_new_entity(id_list=updating_experiments, api_path='https://api.optimizely.com/v2/projects', table='experiment')



In [ ]:
result_endpoint = f'https://api.optimizely.com/v2/experiments/13559460138/timeseries'


In [ ]:
response_ts = requests.get(result_endpoint, headers=headers)